# Processing SST2 NEXAFS

## Imports

In [ ]:
## Imports
import pathlib
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm
from scipy import optimize

sys.path.append('/nsls2/users/alevin/rsoxs_suite/nexafs_analysis')
import nexafs_fxns as nf  # custom nexafs functions used in this notebook

## Define paths & sample name dictionary

In [ ]:
# Define paths
propPath = pathlib.Path('/nsls2/data/sst/proposals/2023-2/pass-311130')
peyPath = propPath.joinpath('PEY_NEXAFS')
outPath = propPath.joinpath('processed_data')
zarrsPath = outPath.joinpath('zarrs')

sample_name_dict = {
    'PEY1': 'Y6_Si',
    'PEY2': 'Y7_Si',
    'PEY3': 'PM6_Si',
    'PEY4': 'PM7_Si',
    'PEY5': 'PM6-Y6_Si',
    'PEY8': 'PM7-Y7_Si',
}

## Load raw sst2 nexafs data

In [ ]:
sample_ids = ['PEY1', 'PEY2', 'PEY3', 'PEY4', 'PEY5', 'PEY8']
column_names = ['energy', 'tey', 'i_ref', 'pey', 'i_0', 'm4c', 'fy_csi']

# Load first sample into DataArray -> Dataset
DS_sample_concat = []
for sample_id in tqdm(sample_ids):
    DS_theta_concat = []
    for f in sorted(list(peyPath.glob(f'*{sample_id}*'))):
        with f.open(mode='r') as file:
            lines = file.readlines()

        theta = int(lines[13].split('/')[1][-2:])
        sample_id = f.name.split('_')[0][2:]
        # columns = lines[17].split()

        d = np.loadtxt(f, skiprows=18)
        energy = d[:,0]

        DS = xr.Dataset(data_vars = {
                            'tey': ('energy', d[:,1]),
                            'i_ref': ('energy', d[:,2]),
                            'pey': ('energy', d[:,3]),
                            'i_0': ('energy', d[:,4]),
                            'm4c': ('energy', d[:,5])
                        },
                        coords = {'energy':energy})

        DS = DS.expand_dims({'theta': [theta], 'sample_id': [sample_id]})

        energies = np.linspace(275, 310, 6000)
        DS = DS.interp({'energy':energies})

        DS_theta_concat.append(DS)

    DS = xr.concat(DS_theta_concat, dim='theta')
    DS_sample_concat.append(DS)
    
DS = xr.concat(DS_sample_concat, dim='sample_id')
DS = (DS.assign_coords({'sample_name':('sample_id', [sample_name_dict[sample_id] for sample_id in DS.sample_id.values]),
                        'cos_sq_theta': ('theta', ((np.cos(np.deg2rad(DS.theta)))**2).data)})
        .swap_dims({'sample_id':'sample_name'}))
DS

## Apply pre and post edge correction

In [ ]:
# Apply pre and post-edge normalizations (more involved right now...):
data_var = 'pey'

# Carbon
pre_edge_energy_slice = slice(276, 280)
post_edge_energy_slice = slice(305, None)

# pre_edge_energy, post_edge_energy = [395, 430] # nitrogen
# pre_edge_energy, post_edge_energy = [525, 555] # oxygen
# pre_edge_energy, post_edge_energy = [680, 720] # oxygen


# First generate list of corrected spectra
pre_corr_list = []
full_corr_list = []
for sample_name in DS.sample_name.values:
    for theta in DS.theta.values:
        # Linear fit for pre-edge region
        # coefficients = np.polyfit(DS.energy.where(DS.energy<pre_edge_energy, drop=True), 
        #                           (DS['dnorm_trmsn']).where(DS.energy<pre_edge_energy, drop=True).sel(sample_name=sample_name, theta=theta), 1)   
        coefficients = np.polyfit(DS.energy.sel(energy=pre_edge_energy_slice), 
                                  DS[data_var].sel(energy=pre_edge_energy_slice, sample_name=sample_name, theta=theta), 
                                  1)
        y_fit = np.polyval(coefficients, DS.energy)
        pre_corr_DA = (DS[data_var].sel(sample_name=sample_name, theta=theta)) - y_fit
        pre_corr_DA = pre_corr_DA.expand_dims({'sample_name': [pre_corr_DA.sample_name.data],
                                               'theta': [pre_corr_DA.theta.data]})
        pre_corr_DA = pre_corr_DA.assign_coords({'sample_id': ('sample_name', [pre_corr_DA.sample_id.data])})     
        
        pre_corr_list.append(pre_corr_DA)

        # Post edge
        # post_edge_mean = corr_DA.where(DS.energy>post_edge_energy, drop=True).mean(dim="energy")
        post_edge_mean = pre_corr_DA.sel(energy=post_edge_energy_slice).mean(dim="energy")
        full_corr_DA = pre_corr_DA / float(post_edge_mean)        
        
        full_corr_list.append(full_corr_DA)

# Now reconstruct full dataarray and add to dataset created in prior cell
corr_names = ['pre_corr_pey', 'full_corr_pey']

for i, corr_list in enumerate([pre_corr_list, full_corr_list]):
# corr_names = ['pre_corr_trmsn']
# for i, corr_list in enumerate([pre_corr_list]):
    # First group by sample_name
    grouped_by_sample = {}
    for corr_DA in corr_list:
        sample_name = corr_DA.sample_name.values[0]
        if sample_name not in grouped_by_sample:
            grouped_by_sample[sample_name] = []
        grouped_by_sample[sample_name].append(corr_DA)

    # Concatenate by theta for each sample_name
    concat_by_theta = {}
    for sample_name, group in grouped_by_sample.items():
        concat_by_theta[sample_name] = xr.concat(group, dim="theta")

    # Finally, concatenate the result along the sample_name dimension
    DS[corr_names[i]] = xr.concat(list(concat_by_theta.values()), dim="sample_name")
    
DS

### Optionally save & load zarr

In [ ]:
DS.to_zarr(zarrsPath.joinpath('PEY_nexafs_carbon.zarr'), mode='w')

In [ ]:
loaded_DS = xr.open_zarr(zarrsPath.joinpath('PEY_nexafs_carbon.zarr'))
DS = loaded_DS.compute()
DS

## Plotting

### NEXAFS all

In [ ]:
energy_min = 275
energy_max = 310
energy_slice = slice(energy_min, energy_max) 

# Create the colormap
colors = plt.cm.plasma_r(np.linspace(0.15, 1, len(DS.theta)))

for sample_name in tqdm(DS.sample_name.values):
    fig, axs = plt.subplots(2, 2)
    fig.set(size_inches=(9, 6), dpi=120, tight_layout=True)
    fig.suptitle(f'PEY carbon-edge NEXAFS: {sample_name}', fontsize=14)

    for i, theta_val in enumerate(DS.theta.values):
        if sample_name=='PM6-Y6_Si' and theta_val==90:
            sliced_DS = DS.sel(energy=energy_slice, sample_name=sample_name, theta=theta_val)
            sliced_DS['pey'].plot.line(ax=axs[0,0], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['i_0'].plot.line(ax=axs[0,1], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['pre_corr_pey'].plot.line(ax=axs[1,0], color=colors[i], label=f'{int(theta_val)}°') 
            # sliced_DS['full_corr_pey'].plot.line(ax=axs[1,1], color=colors[i], label=f'{int(theta_val)}°')
        elif sample_name=='PM7-Y7_Si' and theta_val==55:
            sliced_DS = DS.sel(energy=energy_slice, sample_name=sample_name, theta=theta_val)
            sliced_DS['pey'].plot.line(ax=axs[0,0], color=colors[i], label=f'{int(theta_val)}°')
            # sliced_DS['i_0'].plot.line(ax=axs[0,1], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['pre_corr_pey'].plot.line(ax=axs[1,0], color=colors[i], label=f'{int(theta_val)}°')   
            sliced_DS['full_corr_pey'].plot.line(ax=axs[1,1], color=colors[i], label=f'{int(theta_val)}°')
        else:
            sliced_DS = DS.sel(energy=energy_slice, sample_name=sample_name, theta=theta_val)
            sliced_DS['pey'].plot.line(ax=axs[0,0], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['i_0'].plot.line(ax=axs[0,1], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['pre_corr_pey'].plot.line(ax=axs[1,0], color=colors[i], label=f'{int(theta_val)}°')
            sliced_DS['full_corr_pey'].plot.line(ax=axs[1,1], color=colors[i], label=f'{int(theta_val)}°')
    
    axs[0,0].set(title='Raw beamstop diode current')
    axs[0,1].set(title='I0 (gold mesh); missing calibration diode')
    axs[1,0].set(title='Subtract pre-edge linear fit')
    axs[1,1].set(title='Divide by post-edge mean')

    # Optionally, add legends
    axs[0,0].legend(title=r"$\theta$")
    axs[0,1].legend(title=r"$\theta$")
    axs[1,0].legend(title=r"$\theta$")
    axs[1,1].legend(title=r"$\theta$")
    
    plt.show()
    # fig.savefig(outPath.joinpath('nexafs_plots/pey_nexafs_all', f'{sample_name}_energy{energy_min}-{energy_max}.png'), dpi=120)
    plt.close('all')

### NEXAFS single data variable

In [ ]:
energy_min = 282
energy_max = 292
energy_slice = slice(energy_min, energy_max)

for sample_name in tqdm(DS.sample_name.values):
    colors = plt.cm.plasma_r(np.linspace(0.15,1,len(DS.theta)))
    fig, ax = plt.subplots(figsize=(7,3.5), dpi=120, tight_layout=True)
    for i, theta_val in enumerate(DS.theta.values):
        if sample_name=='PM6-Y6_Si' and theta_val==90:
            pass
        else:
            (DS['full_corr_pey'].sel(sample_name=sample_name, theta=theta_val, energy=energy_slice)
                      .plot.line(ax=ax, color=colors[i], label=f'{int(theta_val)}°'))

    fig.suptitle(f'Raw PEY NEXAFS: {sample_name}', x=0.53)
    ax.set(title='Pre-edge linear fit subtracted & post-edge mean normalized', xlabel='X-ray Energy [eV]', ylabel='Normalized NEXAFS [arb. units]')
    ax.legend(title=r'$\theta$')
    # fig.savefig(outPath.joinpath(f'nexafs_plots/pey_nexafs_full_corr', f'{sample_name}_energy{energy_min}-{energy_max}.png'), dpi=120)
    plt.show()
    plt.close('all')

### Molecular tilt fit

In [ ]:
# Run Stöhr 9.16a tilt fit on energy region of NEXAFS
energy_min = 284
energy_max = 286

# sample_name = 'PM6_3000_dSiN_R'
for sample_name in DS.sample_name.values:
    nf_DA = DS['full_corr_pey'].sel(sample_name=sample_name)

    pi_peak_areas = nf.int_area(energy_min, energy_max, nf_DA)

    bnds = [(0, 90), (0, 20)]
    res = optimize.differential_evolution(
        nf.mse_line_tilt, bounds=bnds, args=(pi_peak_areas, nf_DA.theta.values), tol=1e-6)

    alpha, const = res.x

    # Plot intensities from Stöhr 9.16a (scaled with fitted constant) as line
    # along with measured pi_peak_areas, x-axis is cos_sq_theta:
    fig, axs = plt.subplots(ncols=2, figsize=(10,4.5), dpi=120, tight_layout=True)
    fig.suptitle(str(nf_DA.sample_name.values), y=1.03, fontsize=14)
    axs[0].plot(nf_DA.cos_sq_theta.values[:], const * nf.i_nexafs(alpha, nf_DA.theta.values[:]),
            marker='o', label=f'Stöhr 9.16a: ($\\alpha$={np.round(alpha,2)}, const={np.round(const,2)})',
            clip_on=False, zorder=3)
    axs[0].plot(nf_DA.cos_sq_theta.values[:], pi_peak_areas[:], marker='o',
            label=f'NEXAFS integrated areas', clip_on=False, zorder=4)
    axs[0].set(title='Peak fit', xlabel=r'$cos^2(\theta)$', ylabel='Intensity [arb. units]')
    axs[0].set_xticks(nf_DA.cos_sq_theta.values, minor=True)
    axs[0].set_xlim(left=0)
    axs[0].legend()

    # Add secondary axis showing theta values:
    def forward(x):
        return np.arccos(np.sqrt(x)) * 180 / np.pi

    def inverse(x):
        return np.cos(x * np.pi / 180)**2

    ax2 = axs[0].secondary_xaxis(-0.23, functions=(forward, inverse))
    ax2.set(xlabel=r'$\theta$ [$\degree$]')
    ax2.set_xticks(nf_DA.theta.values)


    colors = plt.cm.plasma_r(np.linspace(0.15,1,len(nf_DA.theta)))
    for i, theta_val in enumerate(nf_DA.theta.values):
        (nf_DA.sel(theta=theta_val, energy=slice(282, 292))
               .plot.line(ax=axs[1], color=colors[i], label=f'{int(theta_val)}°'))

    axs[1].axvline(energy_min, color='grey')
    axs[1].axvline(energy_max, color='grey')
    axs[1].set(title='NEXAFS', xlabel='X-ray Energy [eV]', ylabel='Normalized NEXAFS [arb. units]')
    axs[1].legend(title=r'$\theta$ [$\degree$]', loc='upper left')

    fig.savefig(outPath.joinpath('nexafs_plots/pey_tilt_fit', f'{sample_name}_energy{energy_min}-{energy_max}.png'), dpi=120)

    plt.show()
    plt.close('all')

### Calculate optical constants & binary contrasts

#### Define functions & chemical formulas 

In [ ]:
# Define function to calculate binary contrast
def binary_contrast(n1, n2=None):
    if n2 is None:
        #second component is vacuum
        n2 = xr.zeros_like(n1)
    else:
        #second is interpolated to match the energy array of the first component.  
        n2 = n2.interp(energy=n1.energy)
    return n1.energy**4 * ((n1.delta-n2.delta)**2 + (n1.beta-n2.beta)**2)

# Add chemical formulas as coordinate along 'sample_name' dimension
# Define necessary formula stoichiometries
Y6_formula = 'C82H86F4N8O2S5'
Y7_formula = 'C82H86Cl4N8O2S5'
PM6_formula = 'C68H76F2O2S8'
PM7_formula = 'C68H76Cl2O2S8'

# Make dictionary of sample_name keys and formula values
# chemical_formulas = {
#     'BareSiN_R': 'SiN',
#     'PM6_3000_dSiN_R': PM6_formula,
#     'PM7-Y6_3000_dSiN_R': PM7_formula+Y6_formula,
#     'Y6_3000_dSiN_R': Y6_formula,
#     'Y7_3000_dSiN_R': Y7_formula,
#     'PM7_3000_dSiN_R': PM7_formula,
#     'PM6-Y6_3000_dSiN_R': PM6_formula+Y6_formula,
#     'PM6-Y7_3000_dSiN_R': PM6_formula+Y7_formula,
#     'PM7-Y7_3000_dSiN_R': PM7_formula+Y7_formula,
#     'Y6_2000_dSiN_R': Y6_formula,
#     'Y7_2000_dSiN_R': Y7_formula
# }

chemical_formulas = {
    'PM6_Si': PM6_formula,
    'PM7-Y6_Si': PM7_formula+Y6_formula,
    'Y6_Si': Y6_formula,
    'Y7_Si': Y7_formula,
    'PM7_Si': PM7_formula,
    'PM6-Y6_Si': PM6_formula+Y6_formula,
    'PM6-Y7_Si': PM6_formula+Y7_formula,
    'PM7-Y7_Si': PM7_formula+Y7_formula,
    'A1_Si': Y6_formula,
    'A2_Si': Y6_formula,
    'A3_Si': Y6_formula,
}

# Define function to take iterable of names and return list corresponding of formulas
def names2formula(names):
    formulas = []
    for name in names:
        formula = chemical_formulas[name]
        formulas.append(formula)
    return formulas

# Assign coordinate
DS = DS.assign_coords({'chemical_formula': ('sample_name', names2formula(DS.sample_name.values))})
DS

#### Generate optical constants dataset

In [ ]:
# Calculate optical constants via kkcalc
# Also calculate vacuum contrast
nexafs_type = 'pey'

n_list = []
for sample_name in tqdm(DS.sample_name.values):
    nf_DA = DS[f'full_corr_{nexafs_type}'].sel(sample_name=sample_name)
    chemical_formula = str(nf_DA.chemical_formula.values)
    n = nf.run_kkcalc_a(nf_DA, x_min=280, x_max=305, chemical_formula=chemical_formula, density=1.1)
    n = n.expand_dims({'sample_name':[sample_name]})
    n_list.append(n)
    
n_DS = xr.concat(n_list, dim='sample_name')
n_DS = n_DS.swap_dims({'cos_sq_theta':'theta'})
n_DS['vacuum_contrast'] = binary_contrast(n_DS)
n_DS

#### Plot optical constants and vacuum contrasts

In [ ]:
outPath

In [ ]:
# Plot optical constants and vacuum contrast for each sample 

# Set energy bounds and theta
nexafs_type = 'pey'
energy_min = 280
energy_max = 310
energy_slice = slice(energy_min, energy_max) # carbon
theta = 55

# Plotting loop
for sample_name in tqdm(n_DS.sample_name.values):
    fig, axs = plt.subplots(ncols=2, figsize=(12,4), tight_layout=True)
    fig.suptitle(f'{sample_name}: magic angle optical constants & vacuum contrast from PEY NEXAFS', fontsize=14, x=0.53)
    n_DS['delta'].sel(sample_name=sample_name, theta=theta, energy=energy_slice).plot.line(ax=axs[0], label=r'$\delta$')
    n_DS['beta'].sel(sample_name=sample_name, theta=theta, energy=energy_slice).plot.line(ax=axs[0], label=r'$\beta$')
    n_DS['vacuum_contrast'].sel(sample_name=sample_name, theta=theta, energy=energy_slice).plot.line(ax=axs[1], yscale='log')
    axs[0].legend()
    axs[0].set(title=r'Optical Constants, $\theta = 55°$', xlabel='X-ray energy [eV]', ylabel='Optical Constants Value [arb. units]')
    axs[1].set(title=r'Vacuum Binary Contrast, $\theta = 55°$', xlabel='X-ray energy [eV]', ylabel='Contrast [arb. units]')
    fig.savefig(outPath.joinpath(f'nexafs_plots/{nexafs_type}_carbon_magic_constants', f'{sample_name}_energy{energy_min}-{energy_max}.png'), dpi=120)
    plt.show()
    plt.close('all')

#### Plot binary contrasts

In [ ]:
# Define sample_name pairs to plot binary contrast for
# pairs = [
#     ('PM6_3000_dSiN_R', 'Y6_3000_dSiN_R'),
#     ('PM6_3000_dSiN_R', 'Y6_2000_dSiN_R'),    
#     ('PM6_3000_dSiN_R', 'Y7_3000_dSiN_R'),    
#     ('PM6_3000_dSiN_R', 'Y7_2000_dSiN_R'),    
#     ('PM7_3000_dSiN_R', 'Y6_3000_dSiN_R'),
#     ('PM7_3000_dSiN_R', 'Y6_2000_dSiN_R'),    
#     ('PM7_3000_dSiN_R', 'Y7_3000_dSiN_R'),    
#     ('PM7_3000_dSiN_R', 'Y7_2000_dSiN_R') 
# ]

pairs = [
    ('PM6_Si', 'Y6_Si'),
    ('PM6_Si', 'Y7_Si'),    
    ('PM7_Si', 'Y6_Si'),
    ('PM7_Si', 'Y7_Si'),    
]

# Set energy bounds and select theta
nexafs_type = 'pey'
energy_min = 282
energy_max = 292
energy_slice = slice(energy_min, energy_max)
theta = 55

# Plotting loop
for pair in tqdm(pairs):
    fig, ax = plt.subplots(figsize=(7,4), dpi=120, tight_layout=True)
    (binary_contrast(n_DS.sel(sample_name=pair[0]), n_DS.sel(sample_name=pair[1]))
     .sel(theta=theta, energy=energy_slice)
     .plot.line(ax=ax, yscale='log'))
    ax.set(title=f'Magic angle ($\\theta = 55°$) binary contrast: {pair[0]}:{pair[1]}', ylabel='Contrast [arb. units]', xlabel='X-ray energy [eV]')
    plt.show()
    fig.savefig(outPath.joinpath(f'nexafs_plots/{nexafs_type}_magic_binary_contrasts', f'{pair[0]}-{pair[1]}_energy{energy_min}-{energy_max}.png'), dpi=120)
    plt.close('all')